In [41]:
import pandas as pd
enron = pd.read_csv('/Users/vchintalapati/Downloads/emails.csv')

In [2]:
enron.head(5)

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [6]:
enron['message'][3]

"Message-ID: <13505866.1075863688222.JavaMail.evans@thyme>\nDate: Mon, 23 Oct 2000 06:13:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: randall.gay@enron.com\nSubject: \nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: Randall L Gay\nX-cc: \nX-bcc: \nX-Folder: \\Phillip_Allen_Dec2000\\Notes Folders\\'sent mail\nX-Origin: Allen-P\nX-FileName: pallen.nsf\n\nRandy,\n\n Can you send me a schedule of the salary and level of everyone in the \nscheduling group.  Plus your thoughts on any changes that need to be made.  \n(Patti S for example)\n\nPhillip"

In [ ]:
prompt_head = """
<context>
I will feed you a lot of emails from Enron, which are a mix of both internal and external emails from Enron. 

My goal is to create a high quality dataset which contains the Subject and Body of emails, specifically external emails of Enron.


Your task is to take all the emails I give you, and to output the emails which you believe are external emails, with the subject and body in the below format. Do not manipulate the content of the Subject and Body, just put them in the specific format I shared with you.

{"text":""}

You need to provide the prompts in this JSON format:
{ "simplePrompt": "", "advancedPrompt": "" }
Here is an example:
<examples>
{"text":"<subject>Receipt of Team Selection Form - Executive Impact & Influence</subject><body>Hi Phillip.  We appreciate your prompt attention and completing the Team
Selection information.
Ideally, we needed to receive your team of raters on the Team Selection form
we sent you.  The information needed is then easily transferred into the
database directly from that Excel spreadsheet.  If you do not have the
ability to complete that form, inserting what you listed below, we still
require additional information.
We need each person's email address.  Without the email address, we cannot
email them their internet link and ID to provide feedback for you, nor can
we send them an automatic reminder via email.  It would also be good to have
each person's phone number, in the event we need to reach them.
</body>"}
</examples>

</context>

REMEMBER, your job is to identify the external emails only, and ONLY use them to the output in the requested format. If you think 
"""

In [45]:
import pandas as pd
import re

def extract_subject_and_body_v2(msg):
    # Extract body using the position of "X-FileName:"
    body_start_pos = msg.find("X-FileName:")
    body = msg[body_start_pos:].split("\n\n", 1)[-1] if body_start_pos != -1 else None
    
    # Extract subject using the "Subject:" tag
    subject_match = re.search(r"Subject: (.*?)\n", msg)
    subject = subject_match.group(1) if subject_match else "No Subject Found"
    
    return subject, body

def process_enron_emails(input_filepath, output_filepath=None):
    # Load the CSV from the provided filepath
    enron_df = pd.read_csv(input_filepath)

    # Apply the extraction method to the 'message' column to populate the "Subject" and "Body" columns
    enron_df['Subject'], enron_df['Body'] = zip(*enron_df['message'].apply(extract_subject_and_body_v2))
    
    # If no output filepath is provided, save next to the input file with a "_processed" suffix
    if output_filepath is None:
        output_filepath = input_filepath.replace(".csv", "_processed.csv")
    
    # Save the dataframe with populated "Subject" and "Body" columns to a CSV file
    enron_df.to_csv(output_filepath, index=False)
    
    return output_filepath


In [46]:
input_path = '/Users/vchintalapati/Downloads/emails.csv'
result_path = process_enron_emails(input_path)
print(f"Processed CSV saved at: {result_path}")


Processed CSV saved at: /Users/vchintalapati/Downloads/emails_processed.csv


In [48]:
fixed_enron = pd.read_csv('/Users/vchintalapati/Downloads/emails_processed.csv')

In [49]:
fixed_enron = fixed_enron[['Subject','Body']]

In [50]:
fixed_enron.head(4)

,Subject,Body
0,NaN,Here is our forecast\n\n
1,Re:,Traveling to have a business meeting takes the...
2,Re: test,test successful. way to go!!!
3,NaN,"Randy,\n\n Can you send me a schedule of the s..."


In [57]:
fixed_enron.dropna(inplace=True)

In [58]:
fixed_enron1 = fixed_enron[fixed_enron['Subject'].str.len() >= 20]

In [62]:
fixed_enron2 = fixed_enron1[fixed_enron['Body'].str.len() >= 20]

/var/folders/r5/fwvsqg5511g8mzwgqbptxh5h0000gs/T/ipykernel_73878/323202642.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fixed_enron2 = fixed_enron1[fixed_enron['Body'].str.len() >= 20]


In [71]:
fixed_enron2.head(5)

,Subject,Body,subody
7,Re: PRC review - phone calls,any morning between 10 and 11:30,<subject>7 Re: PR...
8,Re: High Speed Internet Access,1. login: pallen pw: ke9davis\n\n I don't thi...,<subject>7 Re: PR...
9,FW: fixed forward or other Collar floor gas pr...,---------------------- Forwarded by Phillip K ...,<subject>7 Re: PR...
10,Re: FW: fixed forward or other Collar floor ga...,"Mr. Buckner,\n\n For delivered gas behind San ...",<subject>7 Re: PR...
12,Consolidated positions: Issues & To Do list,---------------------- Forwarded by Phillip K ...,<subject>7 Re: PR...


In [81]:
fixed_enron2.drop(columns={'subody'})

,Subject,Body
0,Re: PRC review - phone calls,any morning between 10 and 11:30
1,Re: High Speed Internet Access,1. login: pallen pw: ke9davis\n\n I don't thi...
2,FW: fixed forward or other Collar floor gas pr...,---------------------- Forwarded by Phillip K ...
3,Re: FW: fixed forward or other Collar floor ga...,"Mr. Buckner,\n\n For delivered gas behind San ..."
4,Consolidated positions: Issues & To Do list,---------------------- Forwarded by Phillip K ...
...,...,...
322109,RE: EOL - Multiple Trading Books (URGENT),I will have 4 books:\n\nCAND-MGMT-BAS for all ...
322110,FW: Positions without Sundance,\n\n -----Original Message-----\nFrom: \tReeve...
322111,RE: Counterparty List,"Nicole and everyone, I think the directory is ..."
322112,Trade with John Lavorato,This is a trade with OIL-SPEC-HEDGE-NG (John L...


In [82]:
fixed_enron2 = fixed_enron2.reset_index(drop=True)

In [77]:
# fixed_enron2['subody'] = f"""<subject>{fixed_enron2['Subject']}</subject><body>{fixed_enron2['Body']}</body>"""


In [83]:
fixed_enron2['subody'] = fixed_enron2.apply(lambda row: f"<subject>{row['Subject']}</subject><body>{row['Body']}</body>", axis=1)

In [84]:
fixed_enron2

,Subject,Body,subody
0,Re: PRC review - phone calls,any morning between 10 and 11:30,<subject>Re: PRC review - phone calls</subject...
1,Re: High Speed Internet Access,1. login: pallen pw: ke9davis\n\n I don't thi...,<subject>Re: High Speed Internet Access</subje...
2,FW: fixed forward or other Collar floor gas pr...,---------------------- Forwarded by Phillip K ...,<subject>FW: fixed forward or other Collar flo...
3,Re: FW: fixed forward or other Collar floor ga...,"Mr. Buckner,\n\n For delivered gas behind San ...",<subject>Re: FW: fixed forward or other Collar...
4,Consolidated positions: Issues & To Do list,---------------------- Forwarded by Phillip K ...,<subject>Consolidated positions: Issues & To D...
...,...,...,...
322109,RE: EOL - Multiple Trading Books (URGENT),I will have 4 books:\n\nCAND-MGMT-BAS for all ...,<subject>RE: EOL - Multiple Trading Books (URG...
322110,FW: Positions without Sundance,\n\n -----Original Message-----\nFrom: \tReeve...,<subject>FW: Positions without Sundance</subje...
322111,RE: Counterparty List,"Nicole and everyone, I think the directory is ...",<subject>RE: Counterparty List</subject><body>...
322112,Trade with John Lavorato,This is a trade with OIL-SPEC-HEDGE-NG (John L...,<subject>Trade with John Lavorato</subject><bo...


In [85]:
fixed_enron2['subody'][3]

'<subject>Re: FW: fixed forward or other Collar floor gas price terms</subject><body>Mr. Buckner,\n\n For delivered gas behind San Diego, Enron Energy Services is the appropriate \nEnron entity.  I have forwarded your request to Zarin Imam at EES.  Her phone \nnumber is 713-853-7107.  \n\nPhillip Allen</body>'

In [88]:
fixed_enron2['subody_length']= fixed_enron2.apply(lambda row: len(row['subody']),axis=1)

In [108]:
fixed_enron2

,Subject,Body,subody,subody_length
255094,The Dial Corporation,See attached letter\n,<subject>The Dial Corporation</subject><body>S...,72
267210,The Dial Corporation,See attached letter\n,<subject>The Dial Corporation</subject><body>S...,72
215294,AGA for 9/1/00 is 42,AGA for 9/1/00 is 42,<subject>AGA for 9/1/00 is 42</subject><body>A...,72
222240,Cinergy Letter Draft,Please review. \n\n,<subject>Cinergy Letter Draft</subject><body>P...,72
216867,AGA for 8/4/00 is 65,AGA for 8/4/00 is 65,<subject>AGA for 8/4/00 is 65</subject><body>A...,72
...,...,...,...,...
155559,Undeliverable message,------- Failure Reasons --------\n\nUser not...,<subject>Undeliverable message</subject><body>...,820906
90437,Returned mail: Host unknown (Name server: epec...,"The original message was received at Tue, 6 Ma...",<subject>Returned mail: Host unknown (Name ser...,1697264
90582,Returned mail: Host unknown (Name server: epec...,"The original message was received at Tue, 6 Ma...",<subject>Returned mail: Host unknown (Name ser...,1697264
93030,Returned mail: Host unknown (Name server: epec...,"The original message was received at Tue, 6 Ma...",<subject>Returned mail: Host unknown (Name ser...,1697264


In [111]:
fixed_enron2_sorted = fixed_enron2.sort_values(by='subody_length', ascending=True)


In [112]:
fixed_enron2_sorted

,Subject,Body,subody,subody_length
255094,The Dial Corporation,See attached letter\n,<subject>The Dial Corporation</subject><body>S...,72
267210,The Dial Corporation,See attached letter\n,<subject>The Dial Corporation</subject><body>S...,72
215294,AGA for 9/1/00 is 42,AGA for 9/1/00 is 42,<subject>AGA for 9/1/00 is 42</subject><body>A...,72
222240,Cinergy Letter Draft,Please review. \n\n,<subject>Cinergy Letter Draft</subject><body>P...,72
216867,AGA for 8/4/00 is 65,AGA for 8/4/00 is 65,<subject>AGA for 8/4/00 is 65</subject><body>A...,72
...,...,...,...,...
155559,Undeliverable message,------- Failure Reasons --------\n\nUser not...,<subject>Undeliverable message</subject><body>...,820906
93030,Returned mail: Host unknown (Name server: epec...,"The original message was received at Tue, 6 Ma...",<subject>Returned mail: Host unknown (Name ser...,1697264
90437,Returned mail: Host unknown (Name server: epec...,"The original message was received at Tue, 6 Ma...",<subject>Returned mail: Host unknown (Name ser...,1697264
90582,Returned mail: Host unknown (Name server: epec...,"The original message was received at Tue, 6 Ma...",<subject>Returned mail: Host unknown (Name ser...,1697264


In [120]:
fixed_enron2_sorted = fixed_enron2_sorted.reset_index(drop=True)

In [143]:
print(fixed_enron2_sorted['subody'][145000])

<subject>Thank you for the Charitygift</subject><body>RYI - re:  Executive Committee holiday gift.  This is a thank you from 
Governor Ann Richards' staff.

Rosie
---------------------- Forwarded by Rosalee Fleming/Corp/Enron on 12/13/2000 
11:00 AM ---------------------------

Kathy Mayfield
12/13/2000 10:47 AM


To: Rosalee Fleming/Corp/Enron@ENRON
cc: Elyse Kalmans/Corp/Enron@ENRON 
Subject: Thank you for the Charitygift


---------------------- Forwarded by Kathy Mayfield/Corp/Enron on 12/13/2000 
10:22 AM ---------------------------


smallen@verner.com on 12/13/2000 10:12:39 AM
To: kathy.mayfield@enron.com
cc:  

Subject: Thank you for the Charitygift


Ms. Mayfield,

Thank you for the Charity Gift holiday greeting card. Governor Richards chose 
to donate to Meals on Wheels. On behalf of Governor Richards and Meals on 
Wheels, thank you and best wishes on the New Year.

Sincerely,

Sara Allen
Assistant


</body>


In [136]:
fixed_enron2_sorted['subody_length'][40000]

278

In [144]:
truncated_enron = fixed_enron2_sorted[40000:41000]